In [ ]:
import pandas as pd
import numpy as np
from nltk import sent_tokenize,word_tokenize
from textblob import TextBlob
from nltk import FreqDist
import itertools
from nltk.sentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import nltk
#nltk.download('vader_lexicon')

## Loads pickled data:

In [ ]:
df_first_clean = pd.read_pickle('pickled_data/data_first_clean.pkl')
df_second_clean = pd.read_pickle('pickled_data/data_second_clean.pkl')

## Sentiment analysis 

In [ ]:
#Sentiment analysis on Polarity and Subjectivity for all speeches using TextBlob:
def sentiment(text):
    try:
        return TextBlob(text).sentiment
    except:
        return None
    
df_first_clean['Polarity'] = df_first_clean['first_clean'].apply(sentiment).apply(lambda x: x[0])
df_first_clean['Subjectivity'] = df_first_clean['first_clean'].apply(sentiment).apply(lambda x: x[1])

df_first_clean.to_pickle('pickled_data/data_first_clean_sentiment.pkl')

df_first_clean

### Sentiment analysis based on economic growth


In [ ]:
#Selection of years of economic crisis or growth:
list_economic_crisis = [1930, 1931, 1932, 1986, 1987, 1988, 2008, 2009, 2010]
list_economic_growth = [1926, 1927, 1928, 1992, 1993, 1994, 2005, 2006, 2007]

#### Main method for the analysis

In [ ]:
def method_financial_sentiment(speech):
    #We remove '\n':
    speech = speech.replace('\n', '')
    
    #We split the speech into sentences to optimize for nltk.sentiment:
    list_sentences = speech.split(sep='. ')
    
    #Add all the polarity scores of the sentences to a list:
    scores = []
    
    sentimentIntensityAnalyzer = SentimentIntensityAnalyzer()
    
    for sentence in list_sentences:
        scores.append(sentimentIntensityAnalyzer.polarity_scores(sentence))
    
    #Sums and finds average polarity for each speech:
    list_all_posistive = []
    list_all_neutral = []
    list_all_negative = []
    list_all_compound = []
    
    for score in scores:
        for key, value in score.items():
            if key == 'pos':
                list_all_posistive.append(value)
            if key == 'neu':
                list_all_neutral.append(value)
            if key == 'neg':
                list_all_negative.append(value)
            if key == 'compound':
                list_all_compound.append(value)
                
    avg_posistive = round(sum(list_all_posistive) / len(list_all_posistive), 4)
    avg_neutral = round(sum(list_all_neutral) / len(list_all_neutral), 4)
    avg_negative = round(sum(list_all_negative) / len(list_all_negative), 4)
    avg_compound = round(sum(list_all_compound) / len(list_all_compound), 4)
     
    #Result is a dictionary of the same format as from the original 'SentimentIntensityAnalyzer..polarity_scores()' method:
    result = {'neg': avg_negative, 'neu': avg_neutral, 'pos': avg_posistive, 'compound': avg_compound}
    
    #Return result
    return result

#### Test of the method

In [ ]:
#Simple sentences:
positive_sentence_1 = 'I love CPHBusiness and I think it is the best school'
negative_sentence_1 = 'I hate CPHBusienss and I think it is the worst school'


print('SIMPLE:')
print('Positive:')
print(method_financial_sentiment(positive_sentence_1))
print('Negative:')
print(method_financial_sentiment(negative_sentence_1))


#Complex sentences (from 2002 speech):
positive_sentence_2 = 'America and Afghanistan are now allies against terror. We will be partners in rebuilding that country. And this evening we welcome the distinguished interim leader of a liberated Afghanistan: Chairman Hamid Karzai. The last time we met in this chamber, the mothers and daughters of Afghanistan were captives in their own homes, forbidden from working or going to school. Today women are free, and are part of Afghanistans new government. And we welcome the new Minister of Womens Affairs, Doctor Sima Samar.'
negative_sentence_2 = 'What we have found in Afghanistan confirms that, far from ending there, our war against terror is only beginning. Most of the 19 men who hijacked planes on September the 11th were trained in Afghanistans camps, and so were tens of thousands of others.  Thousands of dangerous killers, schooled in the methods of murder, often supported by outlaw regimes, are now spread throughout the world like ticking time bombs, set to go off without warning. '

print('\nCOMPLEX:')
print('(Slightly) Positive:')
print(method_financial_sentiment(positive_sentence_2))
print('Negative:')
print(method_financial_sentiment(negative_sentence_2))


#Before and after 9/11:
speech_2001 = df_second_clean['speech'][100]
speech_2002 = df_second_clean['speech'][101]

print('\nBEFORE AND AFTER 9/11:')
print('Before (2001):')
print(method_financial_sentiment(speech_2001))
print('After (2002):')
print(method_financial_sentiment(speech_2002))


#### Sentiment analysis using the main method

In [ ]:
# Initializes dataframe and removes unused column (if exists):
df_economic_sentiment = df_second_clean.copy()
if 'second_clean_lemmatized' in df_economic_sentiment.columns:
    del df_economic_sentiment['second_clean_lemmatized']
    
# Uses lists created above to select years of crisis and growth:
df_crisis = df_economic_sentiment.loc[df_economic_sentiment['year'].isin(list_economic_crisis), ['speech']]
df_growth = df_economic_sentiment.loc[df_economic_sentiment['year'].isin(list_economic_growth), ['speech']]

##### Result A: We get one combined result for crisis and growth periods 

In [ ]:
# Combines all speeches from the selected years of crisis and growth:
crisis_speeches = ''.join(df_crisis.speech)
growth_speeches = ''.join(df_growth.speech)

# Uses main method to perform sentiment analysis:
dict_crisis_result = method_financial_sentiment(crisis_speeches)
dict_growth_result = method_financial_sentiment(growth_speeches)

# Print result (plot doesn't make sense here):
print('CRISIS YEARS:')
print(dict_crisis_result)
print('\nGROWTH YEARS:')
print(dict_growth_result)

##### Result B: Results for each speech is added to DataFrame (for potential further analysis)

In [ ]:
# Adds result to main DataFrame:
df_economic_sentiment['sentiment_crisis'] = df_crisis.speech.apply(method_financial_sentiment)
df_economic_sentiment['sentiment_growth'] = df_growth.speech.apply(method_financial_sentiment)

# Removes 'speech' column (if exists):
if 'speech' in df_economic_sentiment.columns:
    del df_economic_sentiment['speech']

# Removes all years from DataFrame that is not part of the analysis:
df_economic_sentiment = df_economic_sentiment.dropna(subset=['sentiment_crisis', 'sentiment_growth'], thresh=1)

#Print result:
df_economic_sentiment

## Text complexity (LIX)


#### Definition of LIX

<b>Calculation:</b>

<img src="./images/lix.png" align=left>

<br><br>

<p>where</p>

<ul>
<li>A is the number of words,</li>
<li>B is the number of periods (defined by period or colon), and</li>
<li>C is the number of long words (more than 6 letters).</li>
</ul>
<p><i>Source: Wikipedia</i><p>
<br>

<b>Scores:</b>
<ul>
    <li>Greater than 54: Very difficult</li>
    <li>Between 45-54 Difficult</li>
    <li>Between 35-44 Medium</li>
    <li>Between 25-34 Easy</li>
    <li>Less than 25: Very easy</li>
</ul>


#### LIX analysis

In [ ]:
# Copy main dataframe (first clean - we don't want lemmatization here!):
df_lix = df_first_clean.copy()
df_lix = df_lix.drop(columns = ['speech', 'first_clean_tokenized'])


# WORD COUNT:
# Method to calculate total words in each speach:
def word_count(speech):
    return sum(1 for word in speech.split(' '))

# Get total words in each speech and add result to DataFrame:
df_lix['word_count'] = df_lix.first_clean.apply(word_count)


# COUNT OF PERIODS:
# Method to calculate total number of periods (period or colon) in each speach:
def period_count(speech):
    result = speech.count('.') + speech.count(':')
    return result

# Get total number of periods in each speech and add result to DataFrame:
df_lix['period_count'] = df_lix.first_clean.apply(period_count)


# COUNT OF LONG WORDS:
# Method to long words (< 6 characters) in each speach:
def long_word_count(speech):
    count = 0
    for word in speech.split(' '):
        if len(word) > 6:
            count += 1
    return count

# Get total number of long words in each speech and add result to DataFrame:
df_lix['long_words_count'] = df_lix.first_clean.apply(long_word_count)


# CALCULATE LIX:
# Calculates LIX for each speech and adds result to DataFrame:
df_lix['lix'] = (df_lix.word_count/df_lix.period_count) + ((df_lix.long_words_count*100) / df_lix.word_count)

#### Present results from LIX analysis:

In [ ]:
# Development in LIX over the years for all speeches:
df_lix.plot(x = 'year', y = 'lix')

In [ ]:
# Development in LIX showing difference between Democrats and Republicans:
df_lix_party = df_lix.copy()

df_lix_party = df_lix.pivot(index='year', columns='Party', values='lix')
df_lix_party.plot(color=['blue', 'red'])

plt.ylabel('LIX')
plt.xlabel('YEARS')
plt.show()

In [ ]:
# Comparison of between speeches performed by Obama and Trump:
mask_obama = (df_lix['President'] == 'Barack Obama')
mask_trump = (df_lix['President'] == 'Donald Trump')

avg_lix_obama = df_lix[mask_obama]['lix'].mean()
avg_lix_trump = df_lix[mask_trump]['lix'].mean()

x = ["Obama", "Trump"]
y = [avg_lix_obama, avg_lix_trump]


plt.bar(x,y)
plt.ylabel('AVG. LIX')
plt.xlabel('PRESIDENT')
plt.show()



## Word frequency 

### Before lemmatization

In [ ]:
#Masks to filter on democrats and republicans
mask_democrats = (df_first_clean['Party'] == 'Democrat')
mask_republicans = (df_first_clean['Party'] == 'Republican')

#DEMOCRATS:
#Filter dataframe on democrats:
df_first_clean_democrats = df_first_clean[mask_democrats]
#Takes 'first_clean_tokenized' column and converts to list:
tokenized_speeches_democrats = df_first_clean_democrats['first_clean_tokenized'].tolist()
#Combines list of lists into one list:
tokenized_speeches_democrats = list(itertools.chain.from_iterable(tokenized_speeches_democrats))

#Republicans:
#Filter dataframe on republicans:
df_first_clean_republicans = df_first_clean[mask_republicans]
#Takes 'first_clean_tokenized' column and converts to list:
tokenized_speeches_republicans = df_first_clean_republicans['first_clean_tokenized'].tolist()
#Combines list of lists into one list:
tokenized_speeches_republicans = list(itertools.chain.from_iterable(tokenized_speeches_republicans))

In [ ]:
#Checks word freq in all democrat speeches:
freqDist_democrats = FreqDist(tokenized_speeches_democrats)

dem_to_dict = dict(freqDist_democrats.most_common(20))

    
plt.bar(*zip(*dem_to_dict.items()), color='blue')
plt.xticks(rotation=90)
plt.title('D - 20 most frequent words')

plt.show()

freqDist_democrats.most_common(20)

In [ ]:
#Checks word freq in all republican speeches:
freqDist_republicans = FreqDist(tokenized_speeches_republicans)

rep_to_dict = dict(freqDist_republicans.most_common(20))

    
plt.bar(*zip(*rep_to_dict.items()), color='red')
plt.xticks(rotation=90)
plt.title('D - 20 most frequent words')

plt.show()

freqDist_republicans.most_common(20)

### After lemmatization

In [ ]:
#Masks to filter on democrats and republicans
mask_democrats = (df_second_clean['Party'] == 'Democrat')
mask_republicans = (df_second_clean['Party'] == 'Republican')

#DEMOCRATS:
#Filter dataframe on democrats:
df_second_clean_democrats = df_second_clean[mask_democrats]
#Takes 'second_clean_lemmatized' column and converts to list:
lemmatized_speeches_democrats = df_second_clean_democrats['second_clean_lemmatized'].tolist()
#Combines list of lists into one list:
lemmatized_speeches_democrats = list(itertools.chain.from_iterable(lemmatized_speeches_democrats))

#Republicans:
#Filter dataframe on republicans:
df_second_clean_republicans = df_second_clean[mask_republicans]
#Takes 'second_clean_lemmatized' column and converts to list:
lemmatized_speeches_republicans = df_second_clean_republicans['second_clean_lemmatized'].tolist()
#Combines list of lists into one list:
lemmatized_speeches_republicans = list(itertools.chain.from_iterable(lemmatized_speeches_republicans))

In [ ]:
#Checks word freq in all democrat speeches:
freqDistr_democrats = FreqDist(lemmatized_speeches_democrats)

dems_to_dict = dict(freqDistr_democrats.most_common(20))

plt.bar(*zip(*dems_to_dict.items()), color='blue')
plt.xticks(rotation=90)
plt.title('D - 20 most frequent words')

plt.show()

freqDist_democrats.most_common(20)

In [ ]:
#Checks word freq in all republican speeches:
freqDistr_republicans = FreqDist(lemmatized_speeches_republicans)

reps_to_dict = dict(freqDistr_republicans.most_common(20))

plt.bar(*zip(*reps_to_dict.items()), color='red')
plt.xticks(rotation=90)
plt.title('D - 20 most frequent words')

plt.show()

freqDist_republicans.most_common(20)